# Lab 6: Random Forest Regression on Malagasy Mammal Traits

### Introduction
 In Lab 5, we used functional trait data on bird species to predict their presence in a given climate type. This week we are using regression tree models on a similar dataset, this one on mammal species.  Our goal is to predict the total number of climate types each mammal species inhabits.

You will compare performance of a single decision tree with a tuned random forest to get a sense of the effectiveness of ensemble methods with optimized hyperparameter values.

There is a fair amount of missing data in this set, which can lead to low reliability of feature splits, increased overfitting, and decreased accuracy of tree models. You will address this issue with a combination of variable omission and imputation.


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

### Step 1: Read data and retain a subset of the columns
Similar to last week, we will need to take some steps to get the data in a usable form. We again reformat the diet-related variable names to match this set: `Diet_Vertebrates`,
    `Diet_Fruits`, `Diet_Flowers`, `Diet_Seeds`, `Diet_Plants`, `Diet_Other`. Then drop columns that are not relevant functional traits or climate type variables.   Next, drop any remaining  variables that have greater than 40% of the observations missing. Print the final dataframe shape. 

In [9]:
dat = pd.read_csv("/courses/EDS232/Data/MamTraitData.csv", encoding="ISO-8859-1").rename(columns={
    'Diet: invertebrates': 'Diet_Invertebrates',
    'Diet: vertebrates': 'Diet_Vertebrates',
    'Diet: fruits': 'Diet_Fruits',
    'Diet: flower/nectar/pollen/gums': 'Diet_Flowers',
    'Diet: seeds': 'Diet_Seeds',
    'Diet: other plant materials': 'Diet_Plants',
    'Diet: scavenge; garbage; carrion; carcasses': 'Diet_Other'
})

In [10]:
pd.set_option('display.max_columns', None)
dat.head()

,AdultBodyMass,CranialCapacity,Diet_Invertebrates,Diet_Vertebrates,Diet_Fruits,Diet_Flowers,Diet_Seeds,Diet_Plants,Diet_Other,HabitatBreadth,ActivityCycle,ForagingStratum,GestationLength,LitterSize,InterbirthInterval,HomeRange,PopulationDensity,SocialGrpSize,Longevity,Dry,Humid,Montane,Subarid,Subhumid
0,78.0,NaN,yes,no,yes,yes,no,yes,no,1.0,2.0,3.0,60.83,1.00,365.0,10.40,18.00,1.0,107.8,0,1,1,0,1
1,1082.0,NaN,no,no,yes,no,no,yes,no,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,1
2,946.0,NaN,no,no,yes,no,no,yes,no,2.0,1.0,3.0,NaN,NaN,NaN,0.02,NaN,NaN,NaN,1,0,0,0,0
3,1180.0,10.0,no,no,yes,no,no,yes,no,2.0,1.0,3.0,110.05,1.01,365.0,0.01,84.85,2.0,NaN,1,1,1,0,1
4,1212.5,NaN,no,no,yes,no,no,yes,no,1.0,1.0,3.0,NaN,NaN,NaN,NaN,114.00,NaN,NaN,0,1,0,1,1


In [12]:
# Drop Columns not relevant to functional traits of climate type variables
# Filter for specified list of variables
dat = dat[["AdultBodyMass", "CranialCapacity", "Diet_Invertebrates", 
           "Diet_Vertebrates", "Diet_Fruits", "Diet_Flowers", 
           "Diet_Seeds", "Diet_Plants", "Diet_Other", 
           "HabitatBreadth", "ActivityCycle", "ForagingStratum", 
           "GestationLength", "LitterSize", "InterbirthInterval", 
           "HomeRange", "PopulationDensity", "SocialGrpSize", 
           "Longevity", "Dry", "Humid", 
           "Montane", "Subarid", "Subhumid"]]

In [3]:
# Drop remaining variables that have greater than 40% observations missing




print(f"Final DataFrame shape: ")

Final DataFrame shape: 


### Step 2: Imputation with KNN
Now impute the values of the missing data in the remaining columns using k-nearest neighbors with `KNNImputer()` for the numeric columns and using the mode for the categorical cariables. Use the 5 nearest neighbors for KNN. This allows us to implement the knn algorithm to predict the missing values for an observation based on similar complete observations. Perform and print a check to ensure that there are no more NA values.

In [ ]:
# Impute with KNN Imputer
knn_impute = KNNImputer(n_neighbors = 5)
X_knn_impute = knn_impute.fit_transform(X_Na)
X_knn_impute = pd.DataFrame(X_knn_impute, columns = X.columns)

# Check to see it worked
X_knn_impute.isna().sum()

### Step 3: Create target variable and numerical encoding 
Create the target variable (`nClimates`) representing the  number of climates in which a species was present. Then finish preparing the data to be used in our models.

- Create a new column `nClimates` that combines the information of the five climate type variables
- Drop the original `Dry`, `Humid`, `Montane`, `Subarid`, and `Subhumid` columns
- Encode categorical variables
- Split the dataset into training and test sets. Use a random state of 808. 

In [ ]:
...

### Step 4: Train a decision tree
Train a single decision tree to predict the number of climate types inhabited by each species.

- Train a `DecisionTreeRegressor`.  Let's apply a constraint on its growth by not allowing the tree to grow further than 5 levels. Use a random state of 808. 
- Make predictions
- Visualize the decision tree using `plot_tree`
- Evaluate and print its performance using mse

In [ ]:
...

Based on the way this tree looks, what would you guess the most important variable to be?

*Your answer here.*

### Step 5: Random forest with hyperparameter tuning
Now we'll use a more sophisticated ensemble approach where we optimize the values of key hyperparameters that govern tree growth.

- Define a parameter grid that includes the follow parameters and values:
  -  (sqrt(p), 6, and no set #) of the number of features to try at each split
  -  (50, 100, 200) total trees (learners)
  -  a maximum tree depth of (3,4,5,6,7)
  -  (2,5,10) minimum samples per split
  -  (1,2,4) minimum samples per leaf
  
- Use `GridSearchCV` with 5-fold cross-validation and mse (`neg_mean_squared_error`) to find the best combination of parameter values
- Train the best estimator.

In [ ]:
...

### Step 6: Make predictions and evaluate random forest model
Use the optimized random forest model (`best_rf`) to predict the number of climates a species inhabits and analyze its performance.

- Use `best_rf` and `predict()` method on `X_test` to generate predictions
- Print the best set of parameter values using thte `best_params` method
- Compute and print the mse to evaluate model performance
- Extract feature importance from the trained random forest model
- Visualize feature importance using a bar plot

In [ ]:
...

Which model had better accuracy, the single tree or the ensemble.  Interpret which traits have the most influence on species' climate adaptability.

*Your answer here.*